In [85]:
import cv2 as cv
import os
import numpy as np
from PIL import Image
import pytesseract

In [90]:
# load image
img = cv.imread('q5Y4XFYojAc.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY);
 # convert to grayscale

otsu_thresh_val = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
high_thresh_val = otsu_thresh_val[0]
lower_thresh_val = otsu_thresh_val[0] * 0.2

#edged = cv.Canny(image, lower_thresh_val, high_thresh_val)
edged = cv.Canny(img, 100, 350)

image, contours, hierarchy = cv.findContours(edged, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv.contourArea, reverse=True)[:1]

def crop_minAreaRect(img, rect):
    # Source: https://stackoverflow.com/questions/37177811/

    # rotate img
    angle = rect[2]
    rows,cols = img.shape[0], img.shape[1]
    matrix = cv.getRotationMatrix2D((cols/2,rows/2),angle,1)
    img_rot = cv.warpAffine(img,matrix,(cols,rows))

    # rotate bounding box
    rect0 = (rect[0], rect[1], 0.0)
    box = cv.boxPoints(rect)
    pts = np.int0(cv.transform(np.array([box]), matrix))[0]
    pts[pts < 0] = 0

    # crop and return
    return img_rot[pts[1][1]:pts[0][1], pts[1][0]:pts[2][0]]

# Find object with the biggest bounding box
mx_rect = (0,0,0,0)      # biggest skewed bounding box
mx_area = 0
for cont in contours:
    arect = cv.minAreaRect(cont)
    area = arect[1][0]*arect[1][1]
    if area > mx_area:
        mx_rect, mx_area = arect, area

# Output to files
roi = crop_minAreaRect(img, mx_rect)
cv.imwrite('Image_crop.jpg', roi)

box = cv.boxPoints(mx_rect)
box = np.int0(box)
cv.drawContours(img,[box],0,(200,0,0),2)
cv.imwrite('Image_cont.jpg', img)

pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open('Image_crop.jpg'), lang='rus', config = tessdata_dir_config))

cv.waitKey(0)
cv.destroyAllWindows()

In [87]:
#img = find_card_file('q5Y4XFYojAc.jpg')
#cv.imshow('img', img)



#img = cv.imread('q5Y4XFYojAc.jpg', 1)
#temp = cv.Canny(img, 100, 340)
#ret,thresh = cv.threshold(temp,128,255,0)
#im2, contours, hierarchy = cv.findContours(thresh,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

#cv.imshow("canny", temp)
#cv.imshow("result_img", im2)

In [ ]:
def find_card_file(image_path):
    return find_card_image(cv.imread(image_path, 0))
#-----------------------------------------------------------------
def find_card_image(image):
    img = _get_magic_card_crop(image)

    return img
#-----------------------------------------------------------------
def _auto_canny_otsu(image):
    otsu_thresh_val = cv.threshold(image, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    high_thresh_val = otsu_thresh_val[0]
    lower_thresh_val = otsu_thresh_val[0] * 0.2

    edged = cv.Canny(image, lower_thresh_val, high_thresh_val)
   # edged = cv.Canny(image, 100, 350)

    
    return edged
#-----------------------------------------------------------------
def _get_magic_card_contour(bw):
    #bw = cv.cvtColor(mat, cv.COLOR_BGR2GRAY)
    edged = _auto_canny_otsu(bw)

    img, contours, hierarchy = cv.findContours(edged, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=cv.contourArea, reverse=True)[:1]

    return contours

#-----------------------------------------------------------------
def _get_magic_card_crop(original_image):
    scale_mult = 0.02
    mat = cv.resize(original_image, (0, 0), fx=scale_mult, fy=scale_mult)
    contour = _get_magic_card_contour(mat)

    # Obtain shape approximation
    card = contour[0]
    peri = cv.arcLength(card, True)
    approx = cv.approxPolyDP(card, 0.02 * peri, True)

    x, y, w, h = cv.boundingRect(card)
    w /= scale_mult
    h /= scale_mult

    w = int(w)
    h = int(h)

    # Test to see if our approximation is a quadrilateral
    if len(approx) != 4:
        rect = rectify(cv.boxPoints(cv.minAreaRect(card)))
    else:
        # Great! We can safely apply our perspective correction technique
        rect = rectify(approx)

    for i in range(4):
        # Compute all 4 corners on original image (re-apply scaling)
        rect[i][0] /= scale_mult
        rect[i][1] /= scale_mult
        
    # Output perspective correction to an image of about the same size
    output = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], np.float32)
    transform = cv.getPerspectiveTransform(rect, output)
    return cv.warpPerspective(original_image, transform, (w, h))
#-----------------------------------------------------------------
def rectify(h):
    """
    Utility code from http://git.io/vGi60A
    Thanks to author of the sudoku example for the wonderful blog posts!
    """

    h = h.reshape((4, 2))
    hnew = np.zeros((4, 2), dtype=np.float32)

    add = h.sum(1)
    hnew[0] = h[np.argmin(add)]
    hnew[2] = h[np.argmax(add)]

    diff = np.diff(h, axis=1)
    hnew[1] = h[np.argmin(diff)]
    hnew[3] = h[np.argmax(diff)]

    return hnew